<a href="https://colab.research.google.com/github/TAUforPython/BioMedAI/blob/main/heartbeat_spectrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#libraries

from tkinter import *
from tkinter import filedialog #work with files
from tkinter import simpledialog #for input
from tkinter.messagebox import showinfo #messages
#for graphs
import matplotlib.pyplot as plt
from math import pi as pi
import math

#root = Tk()
#file opening dialog
fid = filedialog.askopenfilename()
with open(fid,'rb') as f:
    #HEADER
    vers = float(f.read(8)) #file version number
    patientID = f.read(80).decode('utf-8') #patient ID
    patientID = patientID.strip() #trimming spaces at the beginning and end
    recordID = f.read(80).decode('utf-8') #record ID
    recordID = recordID.strip() #trimming spaces at the beginning and end
    startdate = f.read(8).decode('utf-8') #recording start date (dd.mm.yy)
    startdate = startdate.strip() #trimming spaces at the beginning and end
    starttime = f.read(8).decode('utf-8') #recording start time (hh.mm.ss)
    starttime = starttime.strip() #trimming spaces at the beginning and end
    numBytes = int(f.read(8)) #number of bytes
    reserved = f.read(44).decode('utf-8') #reserved
    reserved = reserved.strip() #trimming spaces at the beginning and end
    numRec = int(f.read(8)) #number of records
    duration = float(f.read(8)) #recording duration
    ns = int(f.read(4)) #number of signals
    #an empty label list is created
    label = []
    for ii in range(ns): #label reading
        label.append(f.read(16).decode('utf-8')) #list addition
        label[ii] = label[ii].strip() #trimming spaces at the beginning and end
    #an empty inverter list is created
    transducer = []
    for ii in range(ns): #transducer reading
        transducer.append(f.read(80).decode('utf-8')) #list addition
    #measure units (mV, μV, etc.)
    units = []
    for ii in range(ns): #reading
        units.append(f.read(8).decode('utf-8')) #list addition
        units[ii] = units[ii].strip() #trimming spaces at the beginning and end
    #minimum value
    physicalMin = []
    for ii in range(ns): #reading
        physicalMin.append(float(f.read(8))) #list addition
    #maximum value
    physicalMax = []
    for ii in range(ns): #reading
        physicalMax.append(float(f.read(8))) #list addition
    #minimum value in discrete
    digitalMin = []
    for ii in range(ns): #reading
        digitalMin.append(float(f.read(8))) #list addition
    #maximum value in discrete
    digitalMax = []
    for ii in range(ns): #reading
        digitalMax.append(float(f.read(8))) #list addition
    #prefilter
    prefilter = []
    for ii in range(ns): #reading
        prefilter.append(f.read(80).decode('utf-8')) #list addition
        prefilter[ii] = prefilter[ii].strip() #trimming spaces at the beginning and end
    #samples
    samples = []
    for ii in range(ns): #reading
        samples.append(int(f.read(8))) #list addition
    #signal reserves
    reserv_ns = []
    for ii in range(ns): #reading
        reserv_ns.append(f.read(32).decode('utf-8')) #list addition
        reserv_ns[ii] = reserv_ns[ii].strip() #trimming spaces at the beginning and end
    #calculation the scale factor to translate the discrete in mV
    scalefac = []
    for ii in range(ns):
        sc_tmp = (physicalMax[ii] - physicalMin[ii])/(digitalMax[ii] - digitalMin[ii])
        scalefac.append(sc_tmp)
    #DC component – zero corrector
    dc = []
    for ii in range(ns):
        dc_tmp = physicalMax[ii] - scalefac[ii]*digitalMax[ii]
        dc.append(dc_tmp)
    #data reading: one value “integer with sign” is written in 2 bytes in little order
    #this is when the last byte is signed. Accordingly, the method parameters are set
    #int.from_bytes - read 2 bytes, byteorder='little',signed=True
    data_s = []
    for ii in range (samples[0]):
        data_tmp =  int.from_bytes(f.read(2),byteorder='little',signed=True)
        data_s.append(data_tmp*scalefac[0]+dc[0])

#setting the index range for the entire record
xt = samples[0];
#overview plotting
x=[]
for ii in range (xt):
    x.append(ii)
y=[]
for ii in range (xt):
    y.append(data_s[ii])

#plotting of the 1st signal
plt.plot(x, y, 'b-', linewidth = 0.5, markersize=2)
plt.grid(True)
plt.show()

#index range list
#range_ind=[0,0]
#
root = Tk()
root.withdraw()
showinfo(title='index range input loop',
         message='[begin, end] input, loop exit [0,0]')

#index input loop
while True:
    root = Tk()
    root.withdraw()
    tmp_beg = int(simpledialog.askstring(title='',
                                  prompt='start index'))
    root = Tk()
    root.withdraw()
    tmp_end = int(simpledialog.askstring(title='',
                                  prompt='final index'))

    if tmp_beg >= tmp_end:
         break
    range_beg = tmp_beg
    range_end = tmp_end
    #overview plotting
    x=[]
    for ii in range (range_beg, range_end):
        x.append(ii)
    y=[]
    for ii in range (range_beg,range_end):
        y.append(data_s[ii])
    #plotting of the 1st signal
    plt.plot(x, y,'b-', linewidth = 0.5,markersize=2)
    plt.grid(True)
    plt.show()

#part 2 – actually spectral analysis
#array length calculation
len_t=range_end - range_beg;
#f and t arrays generation
f_diskr=samples[0]/duration
#the number of intervals between samples is 1 fewer
T=(len_t - 1)/f_diskr
#отсчёты времени
t=[]
for ii in range (len_t):
    t.append(ii/f_diskr)
#ECG voltage readings
f=[]
for ii in range (range_beg,range_end):
    f.append(data_s[ii])
# a0
intgr=0
for ii in range (len_t-1):
    intgr=intgr+(f[ii]+f[ii+1])*(t[ii+1]-t[ii])/2
a0=(1/T)*intgr

# n input – number of harmonics
n=int(simpledialog.askstring(title='',
                                  prompt='harmonic number input'))
#setting arrays of Fourier coefficients and frequencies
a=[]
b=[]
wn=[]
#circular frequencies calculation
for ii in range (n):
    wn.append(2*pi*(ii+1)/T)
#Fourier coefficients calculation
for ii in range (n):
    int_cos=0
    int_sin=0
    for jj in range (len_t-1):
        int_cos=int_cos+(math.cos(wn[ii]*t[jj])*f[jj]+
                         math.cos(wn[ii]*t[jj+1])*f[jj+1])*(t[jj+1]-t[jj])/2
        int_sin=int_sin+(math.sin(wn[ii]*t[jj])*f[jj]+
                         math.sin(wn[ii]*t[jj+1])*f[jj+1])*(t[jj+1]-t[jj])/2
    a.append(2*int_cos/T)
    b.append(2*int_sin/T)

#curve reconstruction by coefficients
reb_f=[]
for ii in range (len_t):
    reb_tmp=a0
    for jj in range (n):
        reb_tmp=reb_tmp+a[jj]*math.cos(wn[jj]*t[ii])+b[jj]*math.sin(wn[jj]*t[ii])
    reb_f.append(reb_tmp)

#graphs original and reconstructed by partial Fourier series
#partial Fourier series

plt.plot(x, y,'b-',x,reb_f,'ro',linewidth = 0.5, markersize=2)
plt.grid(True)
plt.show()

#harmonic amplitudes normalized by sum of squares
#that is, relative harmonic energy
sum_gar=0
for ii in range(n):
    sum_gar=sum_gar+pow(a[ii],2)+pow(b[ii],2)
#normalizing multiplier
m_norm=math.pow(sum_gar,0.5)
#amplitude normalization
mag=[]
ngar=[]
for ii in range(n):
    mag.append((pow(a[ii],2)+pow(b[ii],2))/m_norm)
    ngar.append(ii+1)
#histogram plotting
plt.bar(ngar,mag)
plt.grid(True)
plt.show()
